# Run Models (includes RandomForest and XGBoost)

Documentation
https://model.earth/RealityStream  
https://model.earth/RealityStream/input/industries backed-up to Run-Models-bkup.ipynb

Haohao: Loading parameters.yaml file and saving as custom configs on colab user's Google Drive    
https://raw.githubusercontent.com/ModelEarth/RealityStream/main/parameters.yaml

DONE Aashish: Used Pandas for integrated_df (became df) to avoid loading saved .csv files when in Colab at Google.com.  
TO DO (Priority): By each accuracy report, display the model name and the file paths for features and targets.  
TO DO: Add a path parameter that pulls from [all-years](https://colab.research.google.com/drive/1zu0WcCiIJ5X3iN1Hd1KSW4dGn0JuodB8#scrollTo=jxZiI7xcrT4B) generated by our [Industry Features CoLab](https://colab.research.google.com/drive/1HJnuilyEFjBpZLrgxDa4S0diekwMeqnh?usp=sharing)   
TO DO: Allow parameters.yaml path to be set in the left side of CoLab (probably in a Secret). Add info on How To update the left side here.  
TO DO: Load 1 of these 4 bee targets using parameters.yaml setting, remove bees hardcoding from colab  
https://github.com/ModelEarth/RealityStream/tree/main/input/bees/targets  
TO DO: Save small "toy" csv files in the input folders.  
TO DO: Load targets from Google Data Commons by calling a separate python file.  
https://reality.streamlit.app/?parameters=https://raw.githubusercontent.com/ModelEarth/RealityStream/main/parameters.yaml

In [ ]:
import pandas as pd
import regex as re
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import yaml

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_curve, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from xgboost import plot_importance

Mounted at /content/gdrive


In [ ]:
save_files = True # Keep this True -- no files are actually saved presently when setting to true.

# These variables are not in use yet
features_data = "industries"
features_path = "https://raw.githubusercontent.com/ModelEarth/community-timelines/main/training/naics{naics}/US/counties/{year}/US-{state}-training-naics{naics}-counties-{year}.csv"
targets_data = "bees"
targets_path = "https://raw.githubusercontent.com/ModelEarth/RealityStream/main/input/bees/targets/bees-targets-increase2022.csv"
# Irene is removing .0 from FIPS values in .csv file above.

# TO DO: Setting model_name = "XGBoost" resulted in the error:
# ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Population-2018: object, Population-2019: object, Population-2020: object

# These are in use
dataset_name = "bees" # TO DO: eliminate since features and targets will differ.
model_name = "MLP" # Specify the model to be trained
all_model_list = ["LogisticRegression", "SVM", "MLP", "RandomForest", "XGBoost"] # All usable models
assert model_name in all_model_list
valid_report_list = ["RandomForest", "XGBoost"] # All valid models to generate feature-importance report

random_state = 42 # Specify random state

# Feature related information:
country = "US"
years = range(2017,2022)
naics_level = 2
naics_list = [2,4,6]
assert naics_level in naics_list

# Target related information:
target_url = f"https://raw.githubusercontent.com/ModelEarth/RealityStream/main/input/{dataset_name}/targets/{dataset_name}-targets.csv"
target_df = pd.read_csv(target_url) # Get the target csv

if dataset_name == "bees": # Eliminate these lines after switching to parameters.yaml settings
  target_column = '2022_increase' # Specify the target column
  target_list = ['2007_increase','2012_increase','2017_increase','2022_increase'] # Specify all usable target columns
  target_list.remove(target_column) # Drop the one we are interested in

year_list = ["2002","2007","2012","2017","2022"]
drop_list = ['Unnamed: 0','Name','State','State ANSI', 'County ANSI', "Ag District", "Ag District Code"]
all_drop_list = drop_list + target_list + year_list # Drop all columns that can affect the training procedure or are not related

feature_start_idx = 3 # Specify the starting column index in dataset csv for features, where first few columns are for target and id related stuff
target_idx = 0 # Specify the column index for target

# Directory Information:
merged_save_dir = f"../process/{dataset_name}/states-{target_column}-{dataset_name}" # Specify the saving dir for state-separate dataset
full_save_dir = f"../output/{dataset_name}/training" # Specify the saving dir for the integrated dataset

In [ ]:
# Get Dictionaries for state-correspondence and industry-correspondence

STATE_DICT = {
    "AL": "ALABAMA",
    "AK": "ALASKA",
    "AZ": "ARIZONA",
    "AR": "ARKANSAS",
    "CA": "CALIFORNIA",
    "CO": "COLORADO",
    "CT": "CONNECTICUT",
    "DE": "DELAWARE",
    "FL": "FLORIDA",
    "GA": "GEORGIA",
    "HI": "HAWAII",
    "ID": "IDAHO",
    "IL": "ILLINOIS",
    "IN": "INDIANA",
    "IA": "IOWA",
    "KS": "KANSAS",
    "KY": "KENTUCKY",
    "LA": "LOUISIANA",
    "ME": "MAINE",
    "MD": "MARYLAND",
    "MA": "MASSACHUSETTS",
    "MI": "MICHIGAN",
    "MN": "MINNESOTA",
    "MS": "MISSISSIPPI",
    "MO": "MISSOURI",
    "MT": "MONTANA",
    "NE": "NEBRASKA",
    "NV": "NEVADA",
    "NH": "NEW HAMPSHIRE",
    "NJ": "NEW JERSEY",
    "NM": "NEW MEXICO",
    "NY": "NEW YORK",
    "NC": "NORTH CAROLINA",
    "ND": "NORTH DAKOTA",
    "OH": "OHIO",
    "OK": "OKLAHOMA",
    "OR": "OREGON",
    "PA": "PENNSYLVANIA",
    "RI": "RHODE ISLAND",
    "SC": "SOUTH CAROLINA",
    "SD": "SOUTH DAKOTA",
    "TN": "TENNESSEE",
    "TX": "TEXAS",
    "UT": "UTAH",
    "VT": "VERMONT",
    "VA": "VIRGINIA",
    "WA": "WASHINGTON",
    "WV": "WEST VIRGINIA",
    "WI": "WISCONSIN",
    "WY": "WYOMING"
}

try:
    industries_df = pd.read_csv(f"https://raw.githubusercontent.com/ModelEarth/community-data/master/{country.lower()}/id_lists/naics{naics_level}.csv",header=None)
    INDUSTRIES_DICT = industries_df.set_index(0).to_dict()[1]
except:
    INDUSTRIES_DICT = dict()

In [ ]:
def rename_columns(df, year):
    rename_mapping = {}
    for column in df.columns:
      if column not in df.columns[:2]:
          new_column_name = column + f'-{year}'
          rename_mapping[column] = new_column_name

    df.rename(columns=rename_mapping, inplace=True)

def check_directory(directory_path): # Check whether the given directory exists, if not, then create it
    if not os.path.exists(directory_path):
        try:
            os.makedirs(directory_path)
            print(f"Directory '{directory_path}' created successfully.")
        except OSError as e:
            print(f"Error creating directory '{directory_path}': {e}")
    else:
        print(f"Directory '{directory_path}' already exists.")
    return directory_path

In [ ]:
if save_files:
  check_directory(merged_save_dir)

# State-separately, for each state, merging industry features and target on Fips value and County Name, return the merged csv

for state in STATE_DICT:
    data = {}
    for year in years:
        url = f"https://raw.githubusercontent.com/ModelEarth/community-timelines/main/training/naics{naics_level}/{country}/counties/{year}/{country}-{state}-training-naics{naics_level}-counties-{year}.csv"
        data[year] = pd.read_csv(url)
        rename_columns(data[year], year)

    merged_df_feature = pd.merge(data[2017], data[2018], on=['Fips', 'Name'], how='inner')

    for year in range(2019,2022):
        merged_df_feature = pd.merge(merged_df_feature, data[year], on=['Fips', 'Name'], how='inner')

    cols = merged_df_feature.columns.tolist()
    cols = cols[:2] + sorted(cols[2:])
    merged_df_feature = merged_df_feature[cols].rename(columns={"Name": "County"})

    merged_df = pd.merge(merged_df_feature, target_df[target_df["State"]==STATE_DICT[state]], on=["Fips","County"], how="inner")
    merged_df.drop(columns=all_drop_list, axis=1, inplace=True)

    target = merged_df.iloc[:, -1]
    merged_df.drop(columns=[target_column], axis=1, inplace=True)
    merged_df.insert(0, 'target', target)

    if save_files:
      try:
        from google.colab import drive
        drive.mount('/content/drive')
        save_dir = '/content/drive/My Drive/RunModels' #Your Google Drive path
        check_directory(save_dir)

      except ImportError:
        save_dir = merged_save_dir #Use the local directory if not in Google Colab

      file_path = os.path.join(save_dir, f"{state}-{target_column}-{dataset_name}.csv")
      merged_df.to_csv(file_path, index=False)
      print(f"Saved file at: {file_path}")

        #merged_df.to_csv(os.path.join(merged_save_dir, f"{state}-{target_column}-{dataset_name}.csv"), index=False)

if not save_files:
      print(f"Since save_files is false no files are currently saved.")


Directory '../process/bees/states-2022_increase-bees' already exists.
Mounted at /content/drive
Directory '/content/drive/My Drive/RunModels' already exists.
Saved file at: /content/drive/My Drive/RunModels/AL-2022_increase-bees.csv
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory '/content/drive/My Drive/RunModels' already exists.
Saved file at: /content/drive/My Drive/RunModels/AK-2022_increase-bees.csv
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory '/content/drive/My Drive/RunModels' already exists.
Saved file at: /content/drive/My Drive/RunModels/AZ-2022_increase-bees.csv
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory '/content/drive/My Drive/RunModels' already exists.
Saved file at: /content/drive/My Drive/RunMo

In [ ]:
# Integrate all the state-separate csv files into one, return the full dataset csv

if save_files:
  check_directory(full_save_dir)

  dataframes = []
  csv_directory = f"../process/{dataset_name}/states-{target_column}-{dataset_name}"
  csv_files = os.listdir(csv_directory)
  for csv_file in csv_files:
      if csv_file.endswith('.csv'):
          dataframes.append(pd.read_csv(os.path.join(csv_directory, csv_file)))

  integrated_df = pd.concat(dataframes, ignore_index=True)
  df = integrated_df


  if save_files:
    try:
      from google.colab import drive
      drive.mount('/content/drive', force_remount=True)
      save_dir = '/content/drive/My Drive/RunModels' #Your Google Drive path
      check_directory(save_dir)
    except ImportError:
      save_dir = full_save_dir #Use the local directory if not in Google Colab

    file_path = os.path.join(save_dir, f"{target_column}-{dataset_name}.csv")
    integrated_df.to_csv(file_path, index=False)
    print(f"Saved file at: {file_path}")
  #integrated_df.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}.csv"), index=False)

else:
  # TO DO: Use Pandas for integrated_df rather than loading files.
  print(f"# TO DO: Use Pandas for integrated_df rather than loading files.")


Directory '../output/bees/training' created successfully.
Mounted at /content/drive
Directory '/content/drive/My Drive/RunModels' already exists.
Saved file at: /content/drive/My Drive/RunModels/2022_increase-bees.csv


In [ ]:
## Train the model and get the test report
def train_model(model, X_train, y_train, X_test, y_test, over_sample):

    if over_sample:
        sm = SMOTE(random_state = 2)
        X_train, y_train = sm.fit_resample(X_train, y_train.ravel())
        print("Oversampling Done for Training Data.")

    model = model.fit(X_train, y_train)
    print("Model Fitted Successfully.")

    # calculating y_pred
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)

    #roc_auc score
    roc_auc = round(roc_auc_score(y_test, y_pred_prob[:, 1]), 2)
    print(f"\033[1mROC-AUC Score\033[0m \t\t: {roc_auc*100} %")

    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:,1], pos_label=1)

    gmeans = np.sqrt(tpr * (1-fpr))

    ix = np.argmax(gmeans)

    print('\033[1mBest Threshold\033[0m \t\t: %.3f \n\033[1mG-Mean\033[0m \t\t\t: %.3f' % (thresholds[ix], gmeans[ix]))
    best_threshold_num = round(thresholds[ix], 3)

    gmeans_num = round(gmeans[ix], 3)

    y_pred = (y_pred > thresholds[ix])

    #ccuracy score
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_num = f"{accuracy * 100:.1f}"

    print("\033[1mModel Accuracy\033[0m \t\t:", round(accuracy,2,)*100, "%")
    print("\033[1m\nClassification Report:\033[0m")

    #Generate classification report for display and in dictionary for furture report generation
    cfc_report = classification_report(y_test, y_pred)
    cfc_report_dict = classification_report(y_test, y_pred, output_dict= True)
    print(cfc_report)

    return model, y_pred, accuracy_num, gmeans_num, accuracy_num, roc_auc, best_threshold_num, cfc_report_dict

# Train the specified model, impute the nan values, and save the trained model as well as the feature-target report
def train(model_name, target_column, dataset_name, X_train, y_train, X_test, y_test, report_gen, all_model_list, valid_report_list, over_sample=False, model_saving=True, random_state=42):
    assert model_name in all_model_list

    imputer = SimpleImputer(strategy='mean')
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)

    if model_name == "LogisticRegression":
        model = LogisticRegression(max_iter=10000, random_state=random_state)
    elif model_name == "SVM":
        model = SVC(random_state=random_state,probability=True)
    elif model_name == "MLP":
        model = MLPClassifier(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=1000, random_state=random_state)
    elif model_name == "RandomForest":
        model = RandomForestClassifier(n_jobs=3, n_estimators=1000, criterion="gini", random_state=random_state)
        model_fullname = "Random Forest"
    elif model_name == "XGBoost":
        model = xgb.XGBClassifier(random_state=random_state)
        model_fullname = "XGBoost"
    else:
        raise Exception

    if model_name == "XGBoost":
        model, y_pred, accuracy_num, gmeans_num, accuracy_num, roc_auc, best_threshold_num, cfc_report_dict = train_model(model, X_train, y_train, X_test, y_test, over_sample) # No need to impute nan values for XGBoost

    else:
        model, y_pred, accuracy_num, gmeans_num, accuracy_num, roc_auc, best_threshold_num, cfc_report_dict = train_model(model, X_train_imputed, y_train, X_test_imputed, y_test, over_sample)


    save_dir = f"../output/{dataset_name}/saved"
    check_directory(save_dir)

    if model_saving:
        if model_name == "XGBoost":
            save_model(model, None, target_column, dataset_name, model_name, save_dir) # No need to impute nan values for XGBoost
        else:
            save_model(model, imputer, target_column, dataset_name, model_name, save_dir)

    if report_gen:
        if model_name in valid_report_list:
            if model_name == "RandomForest":
                importance_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': model.feature_importances_})
                report = importance_df.sort_values(by='Importance', ascending=False)
            elif model_name == "XGBoost":
                importance_df = pd.DataFrame(list(model.get_booster().get_score().items()), columns=["Feature","Importance"])
                report = importance_df.sort_values(by='Importance', ascending=False)
            else:
                raise Exception

            report["Feature_Name"] = report["Feature"].apply(report_modify)
            report = report.reindex(columns=["Feature","Feature_Name","Importance"])
            report.to_csv(os.path.join(save_dir, f"{target_column}-{dataset_name}-report-{model_name}.csv"), index=False)
        else:
            report = None
            print("No Valid Report for Current Model")

    return model, y_pred, report, model_fullname, cfc_report_dict, accuracy_num, gmeans_num, accuracy_num, roc_auc, best_threshold_num



# Save the trained model and nan-value imputer
def save_model(model, imputer, target_column, dataset_name, model_name, save_dir):
    data = {
    "model": model,
    "imputer": imputer
    }
    with open(os.path.join(save_dir, f"{target_column}-{dataset_name}-trained-{model_name}.pkl"), 'wb') as file:
        pickle.dump(data, file)

# Modify the feature-importance report by adding an industry-correspondence introduction column
def report_modify(value):
    splitted = value.split("-")
    if splitted[0] in ["Emp","Est","Pay"]:
        try:
            modified = splitted[0]+"-"+INDUSTRIES_DICT[splitted[1]]+"-"+splitted[2]
        except:
            modified = value
        return modified
    else:
        return value


def report_generator(cfc_report_dict, model_fullname, model_name, gmeans_num, accuracy_num, roc_auc, best_threshold_num):
    #transfrom report from dictionary to df
    df_report = pd.DataFrame.from_dict(cfc_report_dict).transpose()

    #adjust data display format for md and yaml
    df_report['support'] = df_report['support'].astype(int)
    df_report.iloc[:, 0:3] = df_report.iloc[:, 0:3].round(2)
    df_report.iloc[2,0] = " "
    df_report.iloc[2,1] = " "
    df_report.iloc[2,3] = df_report.iloc[3,3]

    #edit roc_auc format
    roc_auc = roc_auc *100

    #covert numpy float to python float for yaml display
    roc_auc = roc_auc.item()
    best_threshold_num = best_threshold_num.item()
    gmeans_num = gmeans_num.item()

    #markdown file content
    markdown_content = f"""
## {model_fullname} Accuracy

**ROC-AUC Score:** {roc_auc}% &nbsp;&nbsp; **Best Threshold:** {best_threshold_num} &nbsp;&nbsp; **G-Mean:** {gmeans_num} &nbsp;&nbsp; **Model Accuracy:** {accuracy_num}%

                    Precision   Recall      F1-Score    Support

    0               {df_report.iloc[0,0]}        {df_report.iloc[0,1]}        {df_report.iloc[0,2]}        {df_report.iloc[0,3]}
    1               {df_report.iloc[1,0]}        {df_report.iloc[1,1]}        {df_report.iloc[1,2]}        {df_report.iloc[1,3]}

    Accuracy                                {df_report.iloc[2,2]}        {df_report.iloc[3,3]}
    Macro Avg       {df_report.iloc[3,0]}        {df_report.iloc[3,1]}        {df_report.iloc[3,2]}        {df_report.iloc[3,3]}
    Weighted Avg    {df_report.iloc[4,0]}        {df_report.iloc[4,1]}        {df_report.iloc[4,2]}        {df_report.iloc[3,3]}
"""

    #yaml output dictionary
    report_dict = {
    "model_fullname": model_fullname,
    "roc_auc": roc_auc,
    "best_threshold_num": best_threshold_num,
    "gmeans_num": gmeans_num,
    "accuracy_num": accuracy_num,
    "classification_report": df_report.to_dict(orient="index")
    }

    with open(f'{model_name}_accuracy.md','w') as markdown_file:
        markdown_file.write(markdown_content)

    with open(f'{model_name}_accuracy.yaml', "w") as f:
        yaml.dump(report_dict, f, default_flow_style=False)

In [ ]:
# Read the integrated full dataset and do the train-test splitting and save the splitted files
integrated_df = pd.read_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}.csv"))
X_total, y_total = df.iloc[:, feature_start_idx:], df.iloc[:, target_idx] #X_total, y_total = integrated_df.iloc[:, feature_start_idx:], integrated_df.iloc[:, target_idx]
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=random_state)
X_train.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}-X-train.csv"), index=False)
X_test.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}-X-test.csv"), index=False)
y_train.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}-y-train.csv"), index=False)
y_test.to_csv(os.path.join(full_save_dir, f"{target_column}-{dataset_name}-y-test.csv"), index=False)

Model training, testing and results saving:

In [ ]:
#Training Random Forest
model, y_pred, report, model_fullname, cfc_report_dict, accuracy_num, gmeans_num, accuracy_num, roc_auc, best_threshold_num= train("RandomForest", target_column, dataset_name, X_train, y_train, X_test, y_test,
      report_gen=True, all_model_list=all_model_list, valid_report_list=valid_report_list, over_sample=False, model_saving=True, random_state=random_state)

Model Fitted Successfully.
ROC-AUC Score 		: 51.0 %
Best Threshold 		: 0.637 
G-Mean 			: 0.516
Model Accuracy 		: 63.0 %

Classification Report:
              precision    recall  f1-score   support

         0.0       0.32      0.08      0.12       193
         1.0       0.66      0.92      0.77       377

    accuracy                           0.63       570
   macro avg       0.49      0.50      0.45       570
weighted avg       0.54      0.63      0.55       570

Directory '../output/bees/saved' created successfully.


In [ ]:
#Generate markdown and yaml file
report_generator(cfc_report_dict, model_fullname, "RandomForest", gmeans_num, accuracy_num, roc_auc, best_threshold_num)

In [ ]:
#Generating dummy values to handle the categorical columns: 'Population-2018', 'Population-2019', 'Population-2020'
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

#Train XGBoost model
model, y_pred, report, model_fullname, cfc_report_dict, accuracy_num, gmeans_num, accuracy_num, roc_auc, best_threshold_num  = train("XGBoost", target_column, dataset_name, X_train, y_train, X_test, y_test,
      report_gen=True, all_model_list=all_model_list, valid_report_list=valid_report_list, over_sample=False, model_saving=True, random_state=random_state)

Model Fitted Successfully.
ROC-AUC Score 		: 53.0 %
Best Threshold 		: 0.758 
G-Mean 			: 0.534
Model Accuracy 		: 61.0 %

Classification Report:
              precision    recall  f1-score   support

         0.0       0.37      0.23      0.28       193
         1.0       0.67      0.80      0.73       377

    accuracy                           0.61       570
   macro avg       0.52      0.51      0.51       570
weighted avg       0.57      0.61      0.58       570

Directory '../output/bees/saved' already exists.


In [ ]:
#Generate Report for XGBoost
report_generator(cfc_report_dict, "XGBoost", "XGBoost", gmeans_num, accuracy_num, roc_auc, best_threshold_num)